<a href="https://colab.research.google.com/github/cherrydaria/RecoServiceTemplate/blob/HW2/notebooks/Metrics_%26_Visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!git clone -b HW2 https://github.com/cherrydaria/RecoServiceTemplate.git

Cloning into 'RecoServiceTemplate'...
remote: Enumerating objects: 240, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 240 (delta 30), reused 39 (delta 22), pack-reused 187
Receiving objects: 100% (240/240), 126.07 KiB | 4.50 MiB/s, done.
Resolving deltas: 100% (138/138), done.


In [26]:
!ls

data_original  __MACOSX       my_visualization.py  RecoServiceTemplate
__init__.py    my_metrics.py  __pycache__


In [27]:
%cd RecoServiceTemplate

/content/RecoServiceTemplate/functions/RecoServiceTemplate


In [28]:
%cd functions

/content/RecoServiceTemplate/functions/RecoServiceTemplate/functions


In [29]:
!ls

__init__.py  my_metrics.py  my_visualization.py


In [30]:
!pip install rectools

In [31]:
from my_metrics import *

In [32]:
from my_visualization import *

In [33]:
import os
import random
import warnings
import zipfile
import io
from copy import deepcopy
from pprint import pprint

import numpy as np
import pandas as pd

import requests
from tqdm import tqdm

from IPython.display import display
from rectools import Columns
from rectools.dataset import Dataset, Interactions
from rectools.metrics import MAP, NDCG, MeanInvUserFreq, Precision, Recall, Serendipity, MRR
from rectools.metrics.base import MetricAtK
from rectools.model_selection import Splitter, TimeRangeSplitter
from rectools.models import PopularModel, RandomModel
from rectools.models.base import ModelBase

# Подготовка набора данных

In [34]:
url = "https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip"

Инициализация датасета с взаимодействиями

In [35]:
# Выполняем GET-запрос по указанной ссылке
response = requests.get(url)

# Распаковываем архив
with zipfile.ZipFile(io.BytesIO(response.content), "r") as zip_ref:
    # Извлекаем файлы из папки "data_original" в текущую директорию Colab
    zip_ref.extractall()

# Загружаем данные из файлов в соответствующие переменные
items = pd.read_csv("data_original/items.csv")
users = pd.read_csv("data_original/users.csv")

In [36]:
# Загружаем данные о взаимодействиях из файла CSV
interactions = pd.read_csv("data_original/interactions.csv", parse_dates=["last_watch_dt"])

# Переименовываем столбцы для соответствия стандартному формату
interactions.rename(columns={"last_watch_dt": Columns.Datetime, "total_dur": Columns.Weight}, inplace=True)

# Создаем объект класса Interactions из загруженных данных
interactions = Interactions(interactions)

# Выводим первые строки DataFrame для проверки
interactions.df.head()

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250.0,72.0
1,699317,1659,2021-05-29,8317.0,100.0
2,656683,7107,2021-05-09,10.0,0.0
3,864613,7638,2021-07-05,14483.0,100.0
4,964868,9506,2021-04-30,6725.0,100.0


In [37]:
# Выводим первые три строки таблицы items
print("ITEMS:")
display(items.head(3))

# Выводим первые строки таблицы users
print("\nUSERS:")
display(users.head())


ITEMS:


,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."



USERS:


,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0


# Подсчет метрик

In [38]:
k_recos = 10  # Кол-во рекомендаций
n_folds = 3  # Кол-во фолдов для кросс-валидации
random_state = 32  # Семя для инициализации генератора случайных чисел

Cловарь моделей, включающий случайную модель и популярную модель

In [39]:
# Создаем список моделей models
models: list[ModelBase] = [
    RandomModel(random_state=random_state),  # Добавляем модель случайных рекомендаций с указанным random_state
    PopularModel()  # Добавляем популярную модель
]

Словарь метрик

In [40]:
# Создаем пустой словарь metrics для хранения метрик
metrics: dict[str, MetricAtK] = {}

# Для каждого значения k из списка [1, 5, 10] добавляем соответствующие метрики в словарь metrics
for k in [1, 5, 10]:
    metrics.update(
        {
            f"top@{k}_precision": Precision(k=k),               # Метрика точности на top-k рекомендаций
            f"top@{k}_recall": Recall(k=k),                     # Метрика полноты на top-k рекомендаций
            f"top@{k}_ndcg": NDCG(k=k),                       # Метрика NDCG на top-k рекомендаций
            f"top@{k}_map": MAP(k=k),                          # Метрика MAP на top-k рекомендаций
            f"top@{k}_serendipity": Serendipity(k=k),   # Метрика разнообразия на top-k рекомендаций
            f"top@{k}_mean_inv_user_freq": MeanInvUserFreq(k=k),  # Метрика средней инверсии частоты пользователей на top-k рекомендаций
        }
    )

In [41]:
# Создаем пустой список для хранения метрик моделей
model_metrics = []

# Итерируемся по моделям и вычисляем метрики
for model in tqdm(models, total=len(models)):
    # Вычисляем метрики для текущей модели и добавляем их в список model_metrics
    model_metrics.extend(metrics_culc(interactions=interactions, metrics=metrics, model=model, splitter=splitter, k_recos=k_recos))

100%|██████████| 2/2 [01:50<00:00, 55.43s/it]


Визуализация полных метрик

In [20]:
# Визуализация полных метрик для всех моделей
metrics_full_visualization(model_metrics)

# Отображение результатов обучения

In [56]:
# Пример размера истории просмотров
history_size_per_user = 10

# Выбираем популярную модель из списка моделей
model = models[1]

# Создаем датасет на основе взаимодействий пользователей
dataset = Dataset.construct(interactions.df)

# Получаем популярную модель из списка и обучаем ее на датасете
popular_model = models[1]
popular_model.fit(dataset)

# Вызываем функцию для визуализации результатов обучения
visualization_result = training_result_full_visualization(
    model=models[1],  # Передаем выбранную модель
    dataset=Dataset.construct(interactions.df),  # Передаем датасет
    user_ids=[666262, 672861, 955527],  # ID пользователей для анализа
    item_data=items[["item_id", "title", "genres"]],  # Информация о фильмах
    k_recos=k_recos,  # Количество рекомендаций
    interactions=interactions,  # Взаимодействия пользователей
    history_size_per_user=history_size_per_user  # Размер истории просмотров
)

# Отображаем результат
display(visualization_result)


In [58]:
%cd ..

/content/RecoServiceTemplate/functions/RecoServiceTemplate


In [59]:
!mkdir notebooks

In [61]:
!git add notebooks/Metrics_&_Visualization
!git commit -m "Added Metrics_&_Visualization file into notebiiks directory"
!git push origin HW2

/bin/bash: line 1: _Visualization: command not found
fatal: pathspec 'notebooks/Metrics_' did not match any files
Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@f906ec1d3a9b.(none)')
fatal: could not read Username for 'https://github.com': No such device or address


In [62]:
git config --global user.email "tretiakova.d.a@gmail.com"
git config --global user.name "cherrydaria"

SyntaxError: invalid syntax (<ipython-input-62-d2ab507a6f0e>, line 1)